# Projeto 2 - Ciência dos Dados

Nome: Cicero Tiago Carneiro Valentim

Nome: Luiz Felipe Lazzaron

Nome: Marcos Vinícius da Silva

# Introdução

No presente trabalho, visa-se selecionar tweets relevantes para a classificação de postagens que possuem sinais de depressão por parte de seus autores. Nesse contexto, os integrantes dividiram os tweets em três categorias:

1. Irrelavantes ou Neutros. Tweets que não indicam quaisquer indício de depressão;
2. Relevantes. Tweets que possuem relevância na análise de indicação de depressão;
3. Possuem interesse no assunto. Tweets que não demonstram doença mental, mas indicam um interresse no tema e na prevenção de suicídio.

Desse modo, com tal classificação, pode-se desenvolver um marketing específico para os públicos 2 e 3; para os relevantes, pode-se oferecer auxílio e suporte psicológico; para aqueles que possuem interesse no tema "depressão", pode-se direcionar materiais que ensinem a como lidar com pessoas com depressão.

___
# Classificador automático de sentimento


## Preparando o ambiente no jupyter:

In [7]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle

___
## Autenticando no  Twitter

* Conta: ***@MarcosV96118169***

In [8]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @MarcosV96118169

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Etapas do projeto:

### Escolha de um produto e coleta das mensagens


In [9]:
#Produto escolhido:
produto = 'Depressão'

#Quantidade mínima de mensagens capturadas:
n = 500
#Quantidade mínima de mensagens para a base de treinamento:
t = 300

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [10]:
# Cria um objeto para a captura
# api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
# i = 1
# for msg in tweepy.Cursor(api.search, q=produto, lang=lang, tweet_mode="extended").items():
#     new_msg = { 'User Name': msg.user.name,
#                 'Tweet Created At': msg.created_at,
#                 'Tweet Text': msg.full_text.lower(),
#                 'User Location': msg.user.location,
#                 'Phone Type': msg.source,
#                 'Favorite Count': msg.favorite_count,
#                 'Retweets':msg.retweet_count
#                 }
#     if i > n:
#         break

#Embaralhando as mensagens para reduzir um possível viés
# shuffle(msgs)

In [11]:
#Criando um Dataframe com a Base de Dados
# pd.set_option('display.max_colwidth', -1)
# pd.DataFrame(msgs)

Salvando os dados em uma planilha Excel:

In [12]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
# if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
#     writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
#     dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
#     dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

#     dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
#     dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
#     writer.save()

___
### Classificando as mensagens na coragem

Esta etapa é manual. Faça a mesma pelo Excel.

In [13]:
planilha_treinamento = pd.read_excel('depressao.xlsx', "Treinamento")
planilha_teste = pd.read_excel('depressao.xlsx', "Teste")
pd.set_option('display.max_colwidth', -1)

___
# Montando o Classificador Naive-Bayes


Primeiramente, iremos trabalhar com o dataframe treinamento, criando três banco de dados:
    
- **Grupo 1 - Irrelevantes/Neutros**: *group_1*
- **Grupo 2 - Relevantes**: *group_2*
- **Grupo 3 - Interesse no Assunto**: *group_3*


In [14]:
group_1 = planilha_treinamento[planilha_treinamento["Relevância"] == 1]

In [15]:
group_2 = planilha_treinamento[planilha_treinamento["Relevância"] == 2]

In [16]:
group_3 = planilha_treinamento[planilha_treinamento["Relevância"] == 3]

In [17]:
groups = [group_1,group_2,group_3]

# Limpeza e Tratamento dos Dados

In [18]:
#Bibliotecas para a Identificação de Emojis
!pip install emoji
from emoji import UNICODE_EMOJI
import emoji
import re 
import functools
import operator

  Running setup.py bdist_wheel for emoji: started
  Running setup.py bdist_wheel for emoji: finished with status 'done'
  Stored in directory: C:\Users\User\AppData\Local\pip\Cache\wheels\2a\a9\0a\4f8e8cce8074232aba240caca3fade315bb49fac68808d1a9c
Successfully built emoji


In [19]:
def space_cleaning(text):
    punctuation = '[!\-.:?;,''"@/]' 
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    split_emoji = emoji.get_emoji_regexp().split(text_subbed)
    split_whitespace = [substr.split() for substr in split_emoji]
    split = functools.reduce(operator.concat,split_whitespace)
    return split

In [20]:
groups_tweet = []
for group in groups:
    tweet_group = group['Tweet Text'] 
    tweet_group = str(tweet_group)
    tweet_group = space_cleaning(tweet_group)
    groups_tweet.append(tweet_group)

In [21]:
# Removendo os links iniciados com 'http'
for tweet_group in groups_tweet:
    for i in range(len(tweet_group)):
        tweet_group[i] = tweet_group[i].replace('\\n','')
        if 'https' in tweet_group[i]:
            tweet_group[i] = 'http'
    for i in range(tweet_group.count('http')):
        tweet_group.remove('http')   

## Naive Bayes e Laplace Smoothing

In [22]:
#Número de Elementos De todos os grupos sem repetição
total_words = []
def lista_sem_repeticao(LISTA,lista):
    resposta = LISTA
    for elemento in lista:
        if elemento not in resposta:
            resposta.append(elemento)
    return resposta
lista_sem_repeticao(total_words,groups_tweet[0])
lista_sem_repeticao(total_words,groups_tweet[1])
lista_sem_repeticao(total_words,groups_tweet[2])
N =len(total_words)

In [23]:
# Algoritmo de Laplace
def laplace(num_vezes_no_grupo, num_elementos_do_grupo, num_elementos_sem_repeticao):
    probabilidade = (num_vezes_no_grupo + 1 ) / ( num_elementos_do_grupo + num_elementos_sem_repeticao)
    return probabilidade

In [24]:
#Número de vezes que a palavra aparece no grupo
def numero_de_vezes(palavra,grupo):
    n = 0
    for valor in grupo:
        if valor ==  palavra:
            n +=1
    return n

In [25]:
#Calculando a probabilidade para cada palavra
resultados = []
for group in groups_tweet:
    dictionary= {}
    for word in group:
        probabilidade = laplace(numero_de_vezes(word,group),len(group),N)
        dictionary[word] = probabilidade
    resultado = pd.Series(dictionary)
    resultados.append(resultado)

___
## Verificando a performance na Planilha Testes

Agora você deve testar o seu classificador com a base de Testes.

In [26]:
teste = planilha_teste["Tweet Text"]

In [27]:
teste_tweet = []
for tweet in teste:
    tweet = str(tweet)
    tweet = space_cleaning(tweet)
    teste_tweet.append(tweet)

In [28]:
# Removendo os links iniciados com 'http'
for tweet in teste_tweet:
    for i in range(len(tweet)):
        tweet[i] = tweet[i].replace('\\n','')
        if 'https' in tweet_group[i]:
            tweet[i] = 'http'
    for i in range(tweet.count('http')):
        tweet.remove('http')   
for tweet in teste_tweet:
    if tweet == []:
        teste_tweet.remove(tweet)

In [29]:
serie_teste = pd.Series(teste_tweet)

In [30]:
#Função de Análise de Frase
def analisa_frase(frase):
    condicional_grupo1 = 1
    condicional_grupo2 = 1
    condicional_grupo3 = 1
    lista = [condicional_grupo1, condicional_grupo2,condicional_grupo3]
    for palavra in frase:
        if (palavra not in resultados[0].index) or (palavra not in resultados[1].index) or (palavra not in resultados[2].index):
            condicional_grupo1 *= 1
            condicional_grupo2 *= 1
            condicional_grupo3 *= 1
        else:
            condicional_grupo1 *= resultados[0][palavra]
            condicional_grupo2 *= resultados[1][palavra]
            condicional_grupo3 *= resultados[2][palavra]
    if max(lista) == condicional_grupo1:
        return 0
    elif max(lista) == condicional_grupo2:
        return 1
    elif max(lista) == condicional_grupo3:
        return 2
    else:
        return("Deu ruim")

In [31]:
analisa_frase(serie_teste[1])

'Deu ruim'

___
### Concluindo

## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

# Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**

#### Importanto NLTK e RSLP:

In [32]:
import nltk 
from nltk.stem import RSLPStemmer

#### Criando funções necessárias para análise:

Função Tokenize: Essa função serve para quebrarmos nosso texto por palavras, criando desse modo um array com todas as palavras contidas dentro do texto. 

In [38]:
def Tokenize(sentence):
    sentence = sentence.lower()
    sentence = nltk.word_tokenize(sentence)
    return sentence

In [39]:
frase = Tokenize("NÃO PODE DAR ERRADO")

LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  Attempted to load [93mtokenizers/punkt/english.pickle[0m

  Searched in:
    - 'C:\\Users\\User/nltk_data'
    - 'C:\\Users\\User\\Anaconda3\\nltk_data'
    - 'C:\\Users\\User\\Anaconda3\\share\\nltk_data'
    - 'C:\\Users\\User\\Anaconda3\\lib\\nltk_data'
    - 'C:\\Users\\User\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
    - ''
**********************************************************************


In [40]:
print(frase)

NÃO PODE DAR ERRADO
